# Timer

In [4]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 9, 17, 30

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [5]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '30m'
higher_timeframe = '6h'
opt_limit = 50
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 30m/6h, trade type is sell
Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:27<00:00, 13.36s/it]


# Check local statistics

In [ ]:
15/4/7

In [2]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
15,PriceChange,16,98.59,96.06,90.99,90.99,89.30,89.86,90.70,87.32,81.13,87.32,83.94,82.82,87.04,88.17,87.89,88.73,88.45,89.86,87.61,90.99,90.99,92.96,92.39,92.11,-1.60,-2.43,-2.08,-1.92,-2.75,-2.93,-2.73,-2.45,-2.51,-2.92,-2.54,-3.04,-3.70,-4.03,-4.37,-4.42,-3.87,-4.12,-3.91,-4.11,-4.46,-5.50,-5.52,-4.69,355,89.425417,-3.441667,6896.022917,-1221.791667
16,PriceChange,17,98.64,96.20,91.30,91.03,89.40,89.95,90.76,86.96,80.71,87.23,83.42,82.61,86.68,87.77,87.77,88.86,88.32,89.95,87.50,90.76,90.76,92.93,92.39,91.85,-1.59,-2.42,-2.06,-1.89,-2.68,-2.90,-2.70,-2.37,-2.44,-2.90,-2.53,-2.99,-3.66,-4.02,-4.29,-4.36,-3.82,-4.06,-3.84,-4.07,-4.45,-5.45,-5.49,-4.70,368,89.322917,-3.403333,7110.833333,-1252.426667
17,PriceChange,18,98.70,96.09,90.89,91.15,89.32,90.10,90.36,86.72,80.21,86.98,83.07,82.03,86.46,88.02,88.02,89.06,88.54,90.10,87.50,90.89,90.62,92.71,92.45,92.19,-1.58,-2.38,-2.01,-1.82,-2.58,-2.80,-2.52,-2.29,-2.37,-2.66,-2.41,-2.95,-3.35,-3.93,-4.21,-4.08,-3.74,-4.01,-3.77,-4.02,-4.39,-5.38,-5.40,-4.67,384,89.257500,-3.305000,7394.880000,-1269.120000
14,PriceChange,15,98.49,96.98,92.15,90.63,89.43,90.03,90.33,86.40,80.66,87.31,83.38,81.87,86.40,87.61,87.31,88.22,87.92,89.43,87.01,90.63,90.63,92.75,92.15,91.84,-1.61,-2.50,-2.11,-1.93,-2.76,-2.99,-2.77,-2.55,-2.53,-2.99,-2.58,-3.18,-4.01,-4.15,-4.38,-4.43,-3.83,-4.08,-3.98,-4.04,-4.49,-5.46,-5.43,-4.62,331,89.148333,-3.475000,6338.098333,-1150.225000
19,PriceChange,20,98.56,95.69,89.71,91.15,88.76,88.76,89.71,85.65,79.19,86.36,82.54,82.78,85.89,87.80,88.52,89.47,89.47,90.43,88.04,90.91,90.67,93.30,93.06,92.82,-1.56,-2.28,-1.97,-1.80,-2.45,-2.70,-2.42,-2.23,-2.24,-2.54,-2.26,-2.78,-3.13,-3.66,-4.01,-3.91,-3.67,-3.84,-3.57,-4.01,-4.38,-5.34,-5.39,-4.66,418,89.135000,-3.200000,7998.430000,-1337.600000
13,PriceChange,14,98.34,97.01,93.02,91.03,90.03,90.70,90.70,86.05,80.07,87.04,82.39,81.06,86.05,87.71,87.71,88.04,87.38,88.70,86.71,90.37,90.37,92.69,91.69,91.36,-1.65,-2.56,-2.17,-1.97,-2.81,-2.99,-2.80,-2.69,-2.66,-3.03,-2.66,-3.24,-4.06,-4.42,-4.52,-4.66,-3.87,-4.15,-4.11,-4.04,-4.58,-5.52,-5.52,-4.64,301,89.009167,-3.555000,5721.759167,-1070.055000
18,PriceChange,19,98.52,95.57,89.66,90.64,89.16,88.92,89.66,85.71,78.82,86.45,82.51,82.02,85.96,87.68,88.18,89.41,89.16,90.15,87.68,90.64,90.39,93.10,92.86,92.61,-1.55,-2.30,-1.98,-1.81,-2.50,-2.73,-2.43,-2.26,-2.31,-2.58,-2.31,-2.92,-3.24,-3.66,-4.04,-3.97,-3.71,-3.90,-3.61,-4.01,-4.37,-5.31,-5.38,-4.65,406,88.977500,-3.230417,7704.865000,-1311.549167
10,PriceChange,11,98.28,96.55,93.53,90.95,89.66,90.52,91.38,87.93,83.19,85.34,80.60,80.60,84.91,85.78,86.21,87.50,87.07,88.36,87.50,90.95,91.38,91.81,90.95,90.95,-1.71,-2.59,-2.21,-1.99,-2.83,-3.03,-2.86,-2.84,-2.80,-3.14,-2.68,-3.40,-4.15,-4.61,-4.67,-4.79,-3.83,-4.14,-4.12,-4.25,-4.79,-5.69,-5.63,-4.91,232,88.829167,-3.652500,4368.366667,-847.380000
12,PriceChange,13,98.55,96.00,93.09,90.91,90.18,90.55,90.91,86.55,81.82,86.55,81.45,80.36,85.45,87

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770


# Save new config data to config file

In [7]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [7]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)